In [18]:
# Importing required packages

import pandas as pd
import numpy as np
import os

In [ ]:
# Defining file paths

offline_input_path = '/opt/sasdata/dev/PricingEngines/DiscountEngines/Russia/data/raw/offline/'
digital_input_path = 'digital_trns/trans_ids/month_id='
output_path = '/opt/sasdata/dev/PricingEngines/DiscountEngines/Russia/data/processed/offline/'

In [ ]:
# pd.read_parquet(offline_input_path + '20200301/data.parquet')

In [ ]:
%%time
for file in [x for x in os.listdir(offline_input_path) if '202003' in x] :
    
    grouper_cols = ['Master_id', 'pos_busn_dt']
    
    # Loading dumped GDW data
    df = pd.read_parquet(offline_input_path + file + '/data.parquet')
    
    # Merging Store Master file
    item_df = pd.read_excel('Item_Master.xlsx')[['sld_menu_itm_id', 'Parent_id']]
    item_df['Master_id'] = np.where(item_df['Parent_id'].isna(), item_df['sld_menu_itm_id'], item_df['Parent_id'])
    
    del item_df['Parent_id']
    
    df = df.merge(item_df, on=['sld_menu_itm_id'], how='left')
    df['Master_id'] = np.where(df['Master_id'].isna(), df['sld_menu_itm_id'], df['Master_id'])
    
    # Getting Trn key
    df['trn_key'] = df['pos_busn_dt'].astype('str') + '-' + df['mcd_gbal_lcat_id_nu'].astype('stre') + '-' + \
                    df['pos_ord_nu'].astype('str')
        
    print('Donw ith loading data')
    # Altering Cmobo price to get WAP if needed
    if 'WAP' in 'WAP' :
        alc_df = df[(df['cmbo_pren_sld_menu_itm_id'] == -1)].reset_index(drop=True)
        cmbo_df = df[(df['cmbo_pren_sld_menu_itm_id'] != -1)].reset_index(drop=True)
    
        del df
        gc.collect()
        
        cmbo_pr_df = cmbo_df.groupby(['pos_busn_dt', 'mcd_gbal_lcat_id_nu', 'pos_ord_nu', 'cmbo_pren_sld_menu_itm_id'])[['pos_itm_grss_unt_prc_am']].sum().reset_index()
        cmbo_pr_df = cmbo_pr_df.merge(cmbo_df[(cmbo_df['cmbo_pren_sld_menu_itm_id'] == cmbo_df['sld_menu_itm_id'])].groupby(['pos_busn_dt', 
                                                                                               'mcd_gbal_lcat_id_nu', 
                                                                                               'pos_ord_nu', 
                                                                                               'cmbo_pren_sld_menu_itm_id'])[['pos_itm_tot_qt']].sum().reset_index(),
                                      on=['pos_busn_dt', 'mcd_gbal_lcat_id_nu', 'pos_ord_nu', 'cmbo_pren_sld_menu_itm_id'], how='left')
        
        cmbo_pr_df['pos_itm_tot_qt'].fillna(1, inplace=True)
        cmbo_pr_df['cmbo_price'] = cmbo_pr_df['pos_itm_grss_unt_prc_am']/cmbo_pr_df['pos_itm_tot_qt']
        
        del cmbo_pr_df['pos_itm_grss_unt_prc_am']
        del cmbo_pr_df['pos_itm_tot_qt']
        
        cmbo_df = cmbo_df.merge(cmbo_pr_df, on=['pos_busn_dt', 'mcd_gbal_lcat_id_nu', 'pos_ord_nu', 'cmbo_pren_sld_menu_itm_id'], how='left')
        cmbo_df['pos_itm_grss_unt_prc_am'] = cmbo_df['cmbo_price']
        
        del cmbo_df['cmbo_price']
        
        df = pd.concat([alc_df, cmbo_df], axis=0).reset_index(drop=True)
        
        # Removing Child Items
        df = df[(df['cmbo_pren_sld_menu_itm_id'] == -1)|(df['cmbo_pren_sld_menu_itm_id'] == df['sld_menu_itm_id'])].reset_index(drop=True)
    
    else :
        # Removing Child Items
        df = df[(df['cmbo_pren_sld_menu_itm_id'] == -1)|(df['cmbo_pren_sld_menu_itm_id'] == df['sld_menu_itm_id'])].reset_index(drop=True)
        
    print('Donw ith WAP')
    # Loading digital trn data to remove those trns
    if os.path.exists(digital_input_path + file + '/'):
        dig_df = pd.read_parquet(digital_input_path + file + 'part-0.parquet')[['order_key', 'store_number']]
        dig_df['pos_ord_nu'] = dig_df['order_key'].apply(lambda x : x.split(':')[1]).astype('int64')
        
        store_df = pd.read_excel('Store_Master.xlsx')[['mcd_gbal_lcat_id_nu', 'busn_lcat_id_nu']]
        dig_df = dig_df.merge(store_df, left_on=['store_number'], right_on=['busn_lcat_id_nu'], how='inner')
        dig_df['mcd_gbal_lcat_id_nu'] = dig_df['mcd_gbal_lcat_id_nu'].astype('int64')
        
        dig_df['digital_flag'] = 1
        del dig_df['order_key']
        del dig_df['store_number']
        del dig_df['busn_lcat_id_nu']
        
        df = df.merge(dig_df, on=['mcd_gbal_lcat_id_nu', 'pos_ord_nu'], how='left')
        print('Donw ith digital merge')
        
        # Clearing space
        del dig_df
        del store_df
        del item_df
        gc.collect()
        
        if 'Digital' in 'Digital' :
            dig_temp_df = df[(~df['digital_flag'].isna())].reset_index(drop=True)
            
            dig_res_df = dig_temp_df.groupby(grouper_cols)[['pos_itm_tot_qt']].sum().reset_index()
            dig_res_df = dig_res_df.merge(dig_temp_df.groupby(grouper_cols)[['trn_key']].nunique().reset_index(), on=grouper_cols)
            dig_res_df = dig_res_df.merge(dig_temp_df[(dig_temp_df['terr_pos_trn_typ_cd'].isin(['014', '114']))].groupby(grouper_cols)[['trn_key']].nunique().reset_index(),
                                          on=grouper_cols)
            
            # Getting WAP
            dig_temp_df['mult'] = dig_temp_df['pos_itm_grss_unt_prc_am']*dig_temp_df['pos_itm_tot_qt']
            dig_wap_df = dig_temp_df.groupby()[['mult', 'pos_itm_tot_qt']].sum().reset_index()
            dig_wap_df['WAP'] = (dig_wap_df['mult']/dig_wap_df['pos_itm_tot_qt']).round(2)
            del dig_wap_df['mult']
            del dig_wap_df['pos_itm_tot_qt']
            del dig_wap_df['pos_itm_grss_unt_prc_am']
            
            dig_res_df = dig_res_df.merge(dig_wap_df, on=grouper_cols)
            print('Donw ith digital result')
            
        if 'Offline' in 'Offline' :
            off_temp_df = df[(df['digital_flag'].isna())].reset_index(drop=True)
            off_res_df = off_temp_df.groupby(grouper_cols)[['pos_itm_tot_qt']].sum().reset_index()
            off_res_df = off_res_df.merge(off_temp_df.groupby(grouper_cols)[['trn_key']].nunique().reset_index(), on=grouper_cols)
            off_res_df = off_res_df.merge(off_temp_df[(off_temp_df['terr_pos_trn_typ_cd'].isin(['014', '114']))].groupby(grouper_cols)[['trn_key']].nunique().reset_index(),
                                          on=grouper_cols)
            
            # Getting WAP
            off_temp_df['mult'] = off_temp_df['pos_itm_grss_unt_prc_am']*off_temp_df['pos_itm_tot_qt']
            off_wap_df = off_temp_df.groupby(grouper_cols)[['mult', 'pos_itm_tot_qt']].sum().reset_index()
            off_wap_df['WAP'] = (off_wap_df['mult']/off_wap_df['pos_itm_tot_qt']).round(2)
            del off_wap_df['mult']
            del off_wap_df['pos_itm_tot_qt']
            del off_wap_df['pos_itm_grss_unt_prc_am']
            
            
            off_res_df = off_res_df.merge(off_wap_df, on=grouper_cols)
            print('Donw ith offline result')
        
    
    
    break

In [ ]:
dig_res_df

In [14]:
import glob
import os
for month_ in ['202109','202110','202111']:
    path = f'/opt/sasdata/dev/PricingEngines/DiscountEngines/Russia/data/processed/company_wise_dv/offline/ungrouped_data/item_level/sld_menu_itm_id=*/company=*/monthid={month_}'
    list_files=glob.glob(path)
    import shutil
    for folder_ in list_files:
        shutil.rmtree(folder_)

In [ ]:
import glob
import os
for month_ in ['202109','202110','202111']:
    path = f'/opt/sasdata/dev/PricingEngines/DiscountEngines/Russia/data/processed/dv_data/offline/day_level/'
    list_files=glob.glob(path)
    import shutil
    for folder_ in list_files:
        shutil.rmtree(folder_)

In [9]:
list_files[2]

'/opt/sasdata/dev/PricingEngines/DiscountEngines/Russia/data/processed/company_wise_dv/offline/ungrouped_data/item_level/sld_menu_itm_id=8663/company=rost/monthid=202110'

In [11]:
path = f'/opt/sasdata/dev/PricingEngines/DiscountEngines/Russia/data/processed/company_wise_dv/offline/ungrouped_data/item_level/sld_menu_itm_id=*/company=*/monthid={month_}'
list_files=glob.glob(path)

In [13]:
len(list_files)

2841

In [15]:
path = f'/opt/sasdata/dev/PricingEngines/DiscountEngines/Russia/data/processed/dv_data/offline/day_level/'
list_files=glob.glob(path)

In [16]:
list_files

['/opt/sasdata/dev/PricingEngines/DiscountEngines/Russia/data/processed/dv_data/offline/day_level/']

In [23]:
pd.read_parquet('/opt/sasdata/dev/PricingEngines/DiscountEngines/Russia/data/processed/company_wise_dv/offline/day_level/company=mcopco/')

ValueError: Schema in partition[monthid=44] /opt/sasdata/dev/PricingEngines/DiscountEngines/Russia/data/processed/company_wise_dv/offline/day_level/company=mcopco//monthid=202109/dda3e1742c0644beafcf3d386d3ce98b.parquet was different. 
pos_busn_dt: timestamp[ms]
num_trans: int64
metadata
--------
{b'ARROW:schema': b'/////3gCAAAQAAAAAAAKAA4ABgAFAAgACgAAAAABAwAQAAAAAAAKAAwA'
                  b'AAAEAAgACgAAALgBAAAEAAAAAQAAAAwAAAAIAAwABAAIAAgAAAAIAAAA'
                  b'EAAAAAYAAABwYW5kYXMAAIEBAAB7ImluZGV4X2NvbHVtbnMiOiBbXSwg'
                  b'ImNvbHVtbl9pbmRleGVzIjogW10sICJjb2x1bW5zIjogW3sibmFtZSI6'
                  b'ICJwb3NfYnVzbl9kdCIsICJmaWVsZF9uYW1lIjogInBvc19idXNuX2R0'
                  b'IiwgInBhbmRhc190eXBlIjogImRhdGV0aW1lIiwgIm51bXB5X3R5cGUi'
                  b'OiAiZGF0ZXRpbWU2NFtuc10iLCAibWV0YWRhdGEiOiBudWxsfSwgeyJu'
                  b'YW1lIjogIm51bV90cmFucyIsICJmaWVsZF9uYW1lIjogIm51bV90cmFu'
                  b'cyIsICJwYW5kYXNfdHlwZSI6ICJpbnQ2NCIsICJudW1weV90eXBlIjog'
